
heap memory manager in C


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <stdbool.h>
#include <string.h>

#define HEAP_SIZE 1024 * 1024 // 1MB heap
#define SMALL_THRESHOLD 128   // Threshold for small allocations

typedef struct Block {
    size_t size;
    bool free;
    struct Block *next;
} Block;

static uint8_t heap[HEAP_SIZE];
static Block *freeList = (Block *)heap;
static void *low_watermark = NULL;
static void *high_watermark = NULL;

void init_memory_manager() {
    freeList->size = HEAP_SIZE - sizeof(Block);
    freeList->free = true;
    freeList->next = NULL;
    low_watermark = heap;
    high_watermark = heap;
}

void *allocate(size_t size) {
    Block *current = freeList;
    while (current) {
        if (current->free && current->size >= size) {
            size_t remaining = current->size - size - sizeof(Block);
            if (remaining > sizeof(Block)) {
                Block *newBlock = (Block *)((uint8_t *)current + sizeof(Block) + size);
                newBlock->size = remaining;
                newBlock->free = true;
                newBlock->next = current->next;
                current->next = newBlock;
                current->size = size;
            }
            current->free = false;
            void *allocated = (void *)((uint8_t *)current + sizeof(Block));
            if (allocated > high_watermark) {
                high_watermark = allocated;
            }
            return allocated;
        }
        current = current->next;
    }
    return NULL; // Out of memory
}

void deallocate(void *ptr) {
    if (!ptr) return;
    Block *block = (Block *)((uint8_t *)ptr - sizeof(Block));
    block->free = true;
    
    Block *current = freeList;
    while (current && current->next) {
        if (current->free && current->next->free) {
            current->size += sizeof(Block) + current->next->size;
            current->next = current->next->next;
        } else {
            current = current->next;
        }
    }
    if (ptr == high_watermark) {
        high_watermark = low_watermark;
        current = freeList;
        while (current) {
            if (!current->free) {
                high_watermark = (void *)((uint8_t *)current + sizeof(Block) + current->size);
            }
            current = current->next;
        }
    }
}

void print_memory_status() {
    printf("Low Watermark: %p\n", low_watermark);
    printf("High Watermark: %p\n", high_watermark);
    Block *current = freeList;
    while (current) {
        printf("Block at %p - Size: %zu - %s\n", current, current->size, current->free ? "Free" : "Used");
        current = current->next;
    }
}

int main() {
    init_memory_manager();
    
    void *ptr1 = allocate(64);
    void *ptr2 = allocate(256);
    print_memory_status();
    
    deallocate(ptr1);
    print_memory_status();
    
    return 0;
}
